In [ ]:
!nvidia-smi

Sun Aug 28 10:14:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    33W / 250W |   3747MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install datasets transformers huggingface_hub

In [ ]:
import re
import os

from typing import List, Dict, Union, Tuple, NoReturn

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

import transformers
from transformers import AutoTokenizer  # Or BertTokenizer
from transformers import AutoModelForPreTraining  # Or BertForPreTraining for loading pretraining heads
from transformers import AutoModel  # or BertModel, for BERT without pretraining heads
from transformers import BertTokenizerFast, BatchEncoding, PreTrainedTokenizerFast, TrainingArguments, Trainer, AutoModelForSequenceClassification
from datasets import load_metric

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
import torch.nn.functional as F

import gc



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
GDRIVE_PATH:str = '/content/drive/MyDrive'
LABEL_NAMES: List[str] = ['Negativa', 'Neutra', 'Positiva']
LABEL_DICT_CONV_MENCOES : Dict[int,str] = {-5:'Negativa', 0:'Neutra', 5:'Positiva'}

MODEL_PATH: str = os.path.join(GDRIVE_PATH, 'model', 'projeto-final')
MODEL_TRAINED_PATH: str = os.path.join(MODEL_PATH, 'bert')
MODEL_TRAINED_LOG: str = os.path.join(MODEL_TRAINED_PATH, 'trainer.log')

BATCH_SIZE = 16
MAX_LEN = 512

BASE_BERT_MODEL: str = 'neuralmind/bert-base-portuguese-cased'

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL_TRAINED_PATH, num_labels=3)
tokenizer = AutoTokenizer.from_pretrained(BASE_BERT_MODEL, do_lower_case=False)
model.to('cuda')

In [ ]:
teste: List[str] =[ 'Um amigo bndes #Cubano está perguntando se o PT continuará investindo em #Cuba com dinheiro do #BNDES(do povo #brasileiro). Falei pra ele ficar tranquilo, pois #Lula fará trens, portos, aeroportos, e muito mais, em Cuba. Esse item do plano de governo do PT mostra bem isso, correto?',
                   'O Programa Emergencial de Acesso a Crédito, tem como objetivo possibilitar a ampliação do acesso ao crédito para MEIs, micro, pequenas e médias empresas, permitindo a manutenção do emprego e da renda.',
                   'Manda ele pegar o #Metro de Caracas pago pelo #BNDES  e levar pra  #BeloHorizonte. #PTNuncamais',
                   'Falei lá atrás que o #PTNuncaMais aparelhou junto com #Cuba, #Venezuela e mais 13 nações via #BNDES e me perdoem #patriotas não lembrar o nome da #Bolívia que foi sim ajudada pelo #bandido #comunista via #Petrobrás com 3 refinarias a custo zero para aquele país sul americano.',
                   'O Banco Nacional de Desenvolvimento Econômico e Social (#BNDES) adiou de 22 de agosto para o próximo dia 6 de setembro a realização de audiência pública para discutir a desestatização do Porto de Santos.']

In [ ]:
encoding = tokenizer.batch_encode_plus(
      teste,
      add_special_tokens=True,
      max_length=MAX_LEN,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt'
    )

#      'texto': texto,
#      'input_ids': encoding['input_ids'].flatten(),
#      'attention_mask': encoding['attention_mask'].flatten(),
#      'labels': torch.tensor(sentimento, dtype=torch.long)
 

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2329: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
encoding[0].tokens

In [ ]:
encoding = tokenizer.batch_encode_plus(
      teste,
      add_special_tokens=True,
      max_length=MAX_LEN,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt'
    )

model_predict = model(encoding['input_ids'].to('cuda'), encoding['attention_mask'].to('cuda'))
model_predict_converted = F.softmax(model_predict.logits, dim=1).cpu().detach().numpy()
predict = np.argmax(model_predict_converted, axis=1)
for index, t in enumerate(teste):
    print(f' Sentimento: {LABEL_NAMES[predict[index]]} \n {t}')
    print(' ===============================================')

 Sentimento: Negativa 
 Um amigo bndes #Cubano está perguntando se o PT continuará investindo em #Cuba com dinheiro do #BNDES(do povo #brasileiro). Falei pra ele ficar tranquilo, pois #Lula fará trens, portos, aeroportos, e muito mais, em Cuba. Esse item do plano de governo do PT mostra bem isso, correto?
 Sentimento: Positiva 
 O Programa Emergencial de Acesso a Crédito, tem como objetivo possibilitar a ampliação do acesso ao crédito para MEIs, micro, pequenas e médias empresas, permitindo a manutenção do emprego e da renda.
 Sentimento: Neutra 
 Manda ele pegar o #Metro de Caracas pago pelo #BNDES  e levar pra  #BeloHorizonte. #PTNuncamais
 Sentimento: Negativa 
 Falei lá atrás que o #PTNuncaMais aparelhou junto com #Cuba, #Venezuela e mais 13 nações via #BNDES e me perdoem #patriotas não lembrar o nome da #Bolívia que foi sim ajudada pelo #bandido #comunista via #Petrobrás com 3 refinarias a custo zero para aquele país sul americano.
 Sentimento: Neutra 
 O Banco Nacional de Desenvo